In [ ]:
import pandas as pd

user_item_df = pd.read_csv("../data/test/enhanced_user_item_matrix.csv")
user_item_df.set_index("user_id", inplace=True)

display(user_item_df.head())

# Normalize the user-item matrix to have values between 0 and 1
user_item_df = (user_item_df - user_item_df.min()) / (user_item_df.max() - user_item_df.min())
print("After value normalization")
display(user_item_df.head())

,103,109,120,122,128,130,131,133,136,137,...,10430,10436,10457,10462,10500,10506,10519,10552,10589,10595
user_id,,,,,,,,,,,,,,,,,,,,,
14,2555.532627,140.314970,3094.539289,44.252806,37.776082,1843.289617,7314.421552,300.778939,3779.264456,231.953560,...,0.348932,0.965220,1.787169,1.816010,0.150323,1.535256,2.757278,0.143556,1.273362,1.719201
19,3718.820330,101.369064,4272.185290,70.304018,150.913867,1024.511952,1349.438703,541.902511,1063.921700,36.683276,...,0.642896,0.633833,0.586222,1.178295,0.000000,0.977297,1.266322,0.265038,0.928168,1.107873
21,16852.380445,97.407430,7682.336825,63.795972,50.590612,992.839518,1648.987309,1495.629002,2467.902842,29.224358,...,0.896847,0.918930,0.602573,0.995887,1.173871,0.957399,1.148837,0.216699,1.210398,1.713792
23,1007.758681,480.281832,525.592370,40.627711,9.831341,1327.354101,77.418778,1888.869481,2743.614892,51.847397,...,0.577134,0.335534,5.304503,0.610346,0.185161,4.725427,0.338674,0.430445,2.225363,0.000000
24,2054.833248,42.541851,3409.617189,73.852996,320.671484,1256.162999,8476.721248,4936.548945,277.308382,391.289361,...,0.884743,0.578658,0.300125,2.151558,2.311935,1.848424,0.388630,0.103633,0.547944,0.093900


After value normalization


,103,109,120,122,128,130,131,133,136,137,...,10430,10436,10457,10462,10500,10506,10519,10552,10589,10595
user_id,,,,,,,,,,,,,,,,,,,,,
14,0.070823,0.276582,0.332097,0.161941,0.089901,0.294772,0.861619,0.060929,0.281640,0.592793,...,0.027564,0.680711,0.336916,0.169635,0.056888,0.324893,0.379649,0.101264,0.572204,0.413448
19,0.103062,0.193708,0.458479,0.262074,0.453875,0.163836,0.151444,0.109774,0.065761,0.093750,...,0.050786,0.447004,0.110514,0.110065,0.000000,0.206817,0.174360,0.186957,0.417086,0.266431
21,0.467043,0.185278,0.824447,0.237059,0.131126,0.158771,0.187107,0.302971,0.177382,0.074687,...,0.070847,0.648066,0.113597,0.093026,0.444241,0.202606,0.158183,0.152859,0.543910,0.412147
23,0.027929,1.000000,0.056405,0.148007,0.000000,0.212266,0.000000,0.382630,0.199302,0.132504,...,0.045591,0.236632,1.000000,0.057013,0.070073,1.000000,0.046632,0.303634,1.000000,0.000000
24,0.056947,0.068529,0.365910,0.275716,1.000000,0.200881,1.000000,1.000000,0.003223,1.000000,...,0.069890,0.408093,0.056579,0.200978,0.874931,0.391165,0.053510,0.073102,0.246227,0.022582


In [17]:
# Reset index to get user_id as a column
df_triplet = user_item_df.reset_index().melt(
    id_vars="user_id", var_name="video_id", value_name="score"
)

# Filter out zeros (unwatched items)
df_triplet = df_triplet[df_triplet["score"] > 0]

# Optionally, convert user_id and video_id to categorical codes for embedding layers
df_triplet["user_idx"] = df_triplet["user_id"].astype("category").cat.codes
df_triplet["video_idx"] = df_triplet["video_id"].astype("category").cat.codes

In [18]:
import numpy as np

user_indices = df_triplet["user_idx"].values
item_indices = df_triplet["video_idx"].values
scores = df_triplet["score"].values

num_users = df_triplet["user_idx"].nunique()
num_items = df_triplet["video_idx"].nunique()

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dot, Flatten
from tensorflow.keras.models import Model

embedding_dim = 32

user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

user_embedding = Embedding(num_users, embedding_dim, name="user_embedding")(user_input)
item_embedding = Embedding(num_items, embedding_dim, name="item_embedding")(item_input)

user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)

pred = Dot(axes=1)([user_vec, item_vec])

model = Model(inputs=[user_input, item_input], outputs=pred)
model.compile(optimizer="adam", loss="mse")

model.fit([user_indices, item_indices], scores, epochs=10, batch_size=256)

Epoch 1/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.1594
Epoch 2/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0424
Epoch 3/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0396
Epoch 4/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0357
Epoch 5/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0284
Epoch 6/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0226
Epoch 7/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0181
Epoch 8/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0135
Epoch 9/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0096
Epoch 10/10
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0068


In [20]:
# Extract learned embeddings
user_embeddings = model.get_layer("user_embedding").get_weights()[0]
item_embeddings = model.get_layer("item_embedding").get_weights()[0]


# Prediction function
def predict_top_n(user_idx, n=10):
    if user_idx >= len(user_embeddings):
        raise ValueError(f"Unknown user index: {user_idx}")
    user_vec = user_embeddings[user_idx]
    scores = item_embeddings @ user_vec
    top_n_indices = np.argsort(scores)[-n:][::-1]
    top_n_scores = scores[top_n_indices]
    return top_n_indices, top_n_scores


# Test recommendations
users_to_test = [0, 1]
results = {u: list(zip(*predict_top_n(u))) for u in users_to_test}

print(results)

{0: [(np.int64(1710), np.float32(1.0604445)), (np.int64(2980), np.float32(1.0538152)), (np.int64(213), np.float32(1.0416571)), (np.int64(2842), np.float32(1.0303992)), (np.int64(2451), np.float32(1.0290784)), (np.int64(87), np.float32(1.02545)), (np.int64(1788), np.float32(1.0225056)), (np.int64(1861), np.float32(1.0215884)), (np.int64(2450), np.float32(1.0190611)), (np.int64(2789), np.float32(1.0188805))], 1: [(np.int64(2533), np.float32(0.78561056)), (np.int64(1887), np.float32(0.7609506)), (np.int64(2484), np.float32(0.7536543)), (np.int64(115), np.float32(0.74749875)), (np.int64(1353), np.float32(0.7300389)), (np.int64(1401), np.float32(0.7293364)), (np.int64(1971), np.float32(0.71500427)), (np.int64(2967), np.float32(0.7149174)), (np.int64(1444), np.float32(0.7141938)), (np.int64(1397), np.float32(0.7125316))]}
